In [1]:
from ipywidgets import Image
from ipywidgets import ColorPicker, IntSlider, link, AppLayout, HBox
from ipycanvas import RoughCanvas, hold_canvas

from src import *

In [2]:
width = 1000
height = 1000

In [12]:
canvas = RoughCanvas(width=width, height=height)
canvas.fill_style = "#c6574f"

drawing = False
position = None
shape = []


def on_mouse_down(x, y):
    global drawing
    global position
    global shape

    drawing = True
    position = (x, y)
    shape = [position]


def on_mouse_move(x, y):
    global drawing
    global position
    global shape

    if not drawing:
        return

    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)

        position = (x, y)

    shape.append(position)


def on_mouse_up(x, y):
    global drawing
    global position
    global shape

    drawing = False
    
    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)

    seg_pts = findSegmenationPoints(shape)
    for seg_pt in seg_pts:
        canvas.fill_rect(x=seg_pt[0], y=seg_pt[1], width=10)
    
    shape = []


canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

canvas.stroke_style = '#749cb8'

picker = ColorPicker(description='Color:', value='#749cb8')

link((picker, 'value'), (canvas, 'stroke_style'))
link((picker, 'value'), (canvas, 'fill_style'))

HBox((canvas, picker))